# The new `QubitMapper` classes in Qiskit Nature

    Max Rossmannek
    Qiskit Demo Day - Feb 23rd, 2023


Overview: https://github.com/Qiskit/qiskit-nature/issues/967
    
    
Special thanks go to Anthony Gandon (https://github.com/Anthony-Gandon) for his great contributions!

## Preparation

First, let us get a `FermionicOp` for demonstration purposes.

In [1]:
from qiskit_nature.second_q.drivers import PySCFDriver

In [2]:
driver = PySCFDriver()
problem = driver.run()
hamiltonian = problem.hamiltonian.second_q_op()

/home/oss/Files/Dev/Qiskit/qiskit-nature/main/qiskit_nature/second_q/operators/polynomial_tensor.py:242: DeprecationWarning: As of version 0.6.0 the return of unwrapped tensors in the `PolynomialTensor.__getitem__` method is deprecated. No sooner that 3 months after this release, arrays will always be returned as `Tensor` objects. You can switch to the new objects immediately, by setting `qiskit_nature.settings.tensor_unwrapping` to `False`.
  return cast(Union[np.ndarray, SparseArray, Tensor], self[key]).shape[0]


In [3]:
print(hamiltonian)

Fermionic Operator
number spin orbitals=4, number terms=36
  -1.2563390730032498 * ( +_0 -_0 )
+ -0.47189600728114245 * ( +_1 -_1 )
+ -1.2563390730032498 * ( +_2 -_2 )
+ -0.47189600728114245 * ( +_3 -_3 )
+ 0.33785507740175813 * ( +_0 +_0 -_0 -_0 )
+ 0.09046559989211565 * ( +_0 +_0 -_1 -_1 )
+ 0.09046559989211556 * ( +_0 +_1 -_0 -_1 )
+ 0.33229086512764827 * ( +_0 +_1 -_1 -_0 )
+ 0.33785507740175813 * ( +_0 +_2 -_2 -_0 )
+ 0.09046559989211565 * ( +_0 +_2 -_3 -_1 )
+ 0.09046559989211556 * ( +_0 +_3 -_2 -_1 )
+ 0.33229086512764827 * ( +_0 +_3 -_3 -_0 )
+ 0.33229086512764816 * ( +_1 +_0 -_0 -_1 )
+ 0.09046559989211574 * ( +_1 +_0 -_1 -_0 )
+ 0.09046559989211564 * ( +_1 +_1 -_0 -_0 )
+ 0.34928686136600906 * ( +_1 +_1 -_1 -_1 )
+ 0.33229086512764816 * ( +_1 +_2 -_2 -_1 )
+ 0.09046559989211574 * ( +_1 +_2 -_3 -_0 )
+ 0.09046559989211564 * ( +_1 +_3 -_2 -_0 )
+ 0.34928686136600906 * ( +_1 +_3 -_3 -_1 )
+ 0.33785507740175813 * ( +_2 +_0 -_0 -_2 )
+ 0.09046559989211565 * ( +_2 +_0 -_1 -_3 )
+ 0

You could always map this to a qubit operator using one of the mappers in Nature:

In [4]:
from qiskit_nature.second_q.mappers import JordanWignerMapper

In [5]:
mapper = JordanWignerMapper()

In [6]:
print(mapper.map(hamiltonian))

-0.8105479805373271 * IIII
+ 0.1721839326191555 * IIIZ
- 0.2257534922240236 * IIZI
+ 0.1721839326191552 * IZII
- 0.22575349222402366 * ZIII
+ 0.12091263261776633 * IIZZ
+ 0.16892753870087907 * IZIZ
+ 0.045232799946057826 * YYYY
+ 0.045232799946057826 * XXYY
+ 0.045232799946057826 * YYXX
+ 0.045232799946057826 * XXXX
+ 0.1661454325638241 * ZIIZ
+ 0.16614543256382408 * IZZI
+ 0.1746434306830045 * ZIZI
+ 0.12091263261776633 * ZZII


/home/oss/Files/Dev/Qiskit/qiskit-nature/main/qiskit_nature/deprecation.py:298: PauliSumOpDeprecationWarning: PauliSumOp is deprecated as of version 0.6.0 and support for them will be removed no sooner than 3 months after the release. Instead, use SparsePauliOp. You can switch to SparsePauliOp immediately, by setting `qiskit_nature.settings.use_pauli_sum_op` to `False`.
  return func(*args, **kwargs)


## How things used to integrate with the stack:

So far, you always had to wrap every mapper into a `QubitConverter` if you wanted to use it in the rest of the stack:

In [7]:
from qiskit_nature.second_q.mappers import QubitConverter

In [8]:
converter = QubitConverter(mapper)

In [9]:
from qiskit.algorithms.minimum_eigensolvers import NumPyMinimumEigensolver
from qiskit_nature.second_q.algorithms import GroundStateEigensolver

In [10]:
algo = GroundStateEigensolver(converter, NumPyMinimumEigensolver())

In [11]:
print(algo.solve(problem))

=== GROUND STATE ENERGY ===
 
* Electronic ground state energy (Hartree): -1.857275030202
  - computed part:      -1.857275030202
~ Nuclear repulsion energy (Hartree): 0.719968994449
> Total ground state energy (Hartree): -1.137306035753
 
=== MEASURED OBSERVABLES ===
 
  0:  # Particles: 2.000 S: 0.000 S^2: 0.000 M: 0.000
 
=== DIPOLE MOMENTS ===
 
~ Nuclear dipole moment (a.u.): [0.0  0.0  1.3889487]
 
  0: 
  * Electronic dipole moment (a.u.): [0.0  0.0  1.3889487]
    - computed part:      [0.0  0.0  1.3889487]
  > Dipole moment (a.u.): [0.0  0.0  0.0]  Total: 0.0
                 (debye): [0.0  0.0  0.0]  Total: 0.0
 


## How things work now:

Instead, you can now use the new mapper instances **directly**:

In [12]:
algo = GroundStateEigensolver(mapper, NumPyMinimumEigensolver())

In [13]:
print(algo.solve(problem))

=== GROUND STATE ENERGY ===
 
* Electronic ground state energy (Hartree): -1.857275030202
  - computed part:      -1.857275030202
~ Nuclear repulsion energy (Hartree): 0.719968994449
> Total ground state energy (Hartree): -1.137306035753
 
=== MEASURED OBSERVABLES ===
 
  0:  # Particles: 2.000 S: 0.000 S^2: 0.000 M: 0.000
 
=== DIPOLE MOMENTS ===
 
~ Nuclear dipole moment (a.u.): [0.0  0.0  1.3889487]
 
  0: 
  * Electronic dipole moment (a.u.): [0.0  0.0  1.3889487]
    - computed part:      [0.0  0.0  1.3889487]
  > Dipole moment (a.u.): [0.0  0.0  0.0]  Total: 0.0
                 (debye): [0.0  0.0  0.0]  Total: 0.0
 


## What other features do the new `QubitMapper` classes have in store?

### 2-qubit reduction

The 2-qubit reduction method has now been directly integrated into the `ParityMapper`:

In [14]:
from qiskit_nature.second_q.mappers import ParityMapper

In [15]:
mapper = ParityMapper(num_particles=(1, 1))

In [16]:
print(mapper.map(hamiltonian))

-1.0523732457728594 * II
+ 0.39793742484317896 * IZ
- 0.39793742484317873 * ZI
- 0.01128010425623538 * ZZ
+ 0.18093119978423122 * XX


### Tapering more qubits

Tapering more qubits is now handled by a separate mapper, the `TaperedQubitMapper`.
Because this relies on problem specific information to find the correct symmetry sector, the easiest way to construct this mapper is like so:

In [17]:
tapered_mapper = problem.get_tapered_mapper(mapper)

In [18]:
print(type(tapered_mapper))

<class 'qiskit_nature.second_q.mappers.tapered_qubit_mapper.TaperedQubitMapper'>


In [19]:
print(tapered_mapper.map(hamiltonian))

-1.0410931415166238 * I
- 0.7958748496863575 * Z
- 0.1809311997842312 * X


If you want to learn how to construct such a mapper manually from the `Z2Symmetries` found in your hamiltonian, be sure to check out the documentation/tutorial once Qiskit Nature 0.6 releases!

### Bonus: interleaved qubit ordering

Qiskit Nature always arranges the qubit register in _blocked_ order, meaning that the fermionic modes corresponding to alpha-spin (or up-spin) particles occupy the first half, and the beta-spin (or down-spin) particles the second half.
We can see this very easily when looking at the `HartreeFock` initial state:

In [20]:
from qiskit_nature.second_q.circuit.library import HartreeFock

In [21]:
hf_state = HartreeFock(2, (1, 1), JordanWignerMapper())
hf_state.draw()

┌───┐
q_0: ┤ X ├
     └───┘
q_1: ─────
     ┌───┐
q_2: ┤ X ├
     └───┘
q_3: ─────

However, sometimes you want to rearrange your qubits into an interleaved layout where the alpha- and beta-spin modes alternate. This can be useful for various circuit designs. For simplicity, let us just look at the `HartreeFock` initial state again:

In [22]:
from qiskit_nature.second_q.mappers import InterleavedQubitMapper

In [23]:
interleaved_mapper = InterleavedQubitMapper(JordanWignerMapper())

In [24]:
hf_state = HartreeFock(2, (1, 1), interleaved_mapper)
hf_state.draw()

┌───┐
q_0: ┤ X ├
     ├───┤
q_1: ┤ X ├
     └───┘
q_2: ─────
          
q_3: ─────

### Yet another bonus: replacing `PauliSumOp` with `SparsePauliOp`

In [25]:
print(type(mapper.map(hamiltonian)))

<class 'qiskit.opflow.primitive_ops.pauli_sum_op.PauliSumOp'>


In [26]:
from qiskit_nature import settings

settings.use_pauli_sum_op = False

In [27]:
print(type(mapper.map(hamiltonian)))

<class 'qiskit.quantum_info.operators.symplectic.sparse_pauli_op.SparsePauliOp'>


In [28]:
import qiskit.tools.jupyter

%qiskit_version_table
%qiskit_copyright